**Applied Machine Learning - Homework 4 - Task1**

Amaury Sudrie (UNI: AS5961)
Maxime Tchibozo (UNI: MT3390)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("drive/My Drive/AML/")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

In [0]:
df = pd.read_csv('winemag-data-130k-v2.csv')

In [0]:
df.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


## **Question 1.1**
Create a baseline model for predicting wine quality using only non-text features

### **Data exploration**
First let's have a look at the dataset. We can directly remove the column **Unamed 0:** as it is just duplicate of indexes. 

In [0]:
df = df.drop(columns="Unnamed: 0")

Let's have a look at the distribution of NA values in the dataset. We can see that except for **region_2** most features have an acceptable number of missing values.

In [0]:
percent_missing = df.isnull().sum() * 100 / len(df)
percent_missing.reset_index().rename(columns={0:'feature',1:'percentage'}, inplace=True)
percent_missing

country                   0.048472
description               0.000000
designation              28.825661
points                    0.000000
price                     6.921544
province                  0.048472
region_1                 16.347493
region_2                 61.136715
taster_name              20.192197
taster_twitter_handle    24.015357
title                     0.000000
variety                   0.000769
winery                    0.000000
dtype: float64

Let's now have a look at unique values. Due to their high cardinality, we can immediately see that **description**, **designation** and **title** will be embedded using Bag of Words.

We can see that **region_2** has only 18 distinct values. Even though it has 60% of missing values, we will keep it.

As **points** and **price** do not contain many values we will not plot the histogram or perform an in-depth analysis of the distribution of these features.

In [0]:
for key in df.keys():
  print(key, len(df[key].unique()))

country 44
description 119955
designation 37980
points 21
price 391
province 426
region_1 1230
region_2 18
taster_name 20
taster_twitter_handle 16
title 118840
variety 708
winery 16757


In [0]:
X,y = df.drop(['points'],axis=1),df['points']

### **Non-text Baseline Model**

From the previous analysis we will represent:

*   **price** and **points** as continuous variables
*   Use Target Encoding for **region_1**, **province**, **winery** and **variety**
*   Use One Hot Encoding for other categorical variables : **country**, **region_2**, **taster_name** and **taster_twitter_handle**


In [0]:
!pip install category_encoders

In [0]:
from sklearn.compose import make_column_transformer
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, Normalizer
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from category_encoders.target_encoder import TargetEncoder

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [0]:
cont = ["price"]
cat_TE = ["province", "region_1", "variety", "winery"]
cat_OHE = ["country", "region_2", "taster_name", "taster_twitter_handle"]

In [0]:
X1 = X_train[cont + cat_OHE + cat_TE]
X1[cat_OHE + cat_TE] = X1[cat_OHE + cat_TE].fillna('missing')
Cont_Imputer = SimpleImputer(strategy='median')
X1[cont] = Cont_Imputer.fit_transform(X1[cont])

X1_test = X_test[cont + cat_OHE + cat_TE]
X1_test[cat_OHE + cat_TE] = X1_test[cat_OHE + cat_TE].fillna('missing')
X1_test[cont] = Cont_Imputer.transform(X1_test[cont])

In [0]:
preprocess = make_column_transformer(
    (TargetEncoder(), cat_TE),
    (OneHotEncoder(handle_unknown='ignore'), cat_OHE),
    (StandardScaler(), cont)
)

model_LR = make_pipeline(preprocess, LinearRegression())

scores = cross_val_score(model_LR, X1, y_train)
model_LR.fit(X1, y_train)
print("Score Linear Regression", np.mean(scores))

Score Linear Regression 0.43124434959959695


In [0]:
print("Score on test", model_LR.score(X1_test,y_test))

Score on test 0.4357303949987641


We can see that a first very simple Linear Regression model using non-text features yields 43% accuracy on both the train and the test sets.

## **Question 1.2**
Create a simple text-based model using a bag-of-words approach and a linear model.

In [0]:
cat_BoW = ["description", "designation", "title"]
X2 = X_train[cat_BoW]
X2 = X2.fillna('missing')

X2_test = X_test[cat_BoW]
X2_test = X2_test.fillna('missing')

In [0]:
vect1 = CountVectorizer(min_df=2, max_features=2000)
vect2 = CountVectorizer(min_df=2, max_features=1000, max_df=0.95)
vect3 = CountVectorizer(min_df=2, max_features=1000, max_df=0.95)

BoW_train = sp.sparse.hstack([
    vect1.fit_transform(X2["description"]),
    vect2.fit_transform(X2["designation"]),
    vect3.fit_transform(X2["title"]),  
])

BoW_train

<97478x4000 sparse matrix of type '<class 'numpy.longlong'>'
	with 3755908 stored elements in COOrdinate format>

In [0]:
LR = LinearRegression()
scores = cross_val_score(LR, BoW_train, y_train)
LR.fit(BoW_train,y_train)
print("Score Linear Regression", np.mean(scores))

Score Linear Regression 0.7185574489294385


In [0]:
BoW_test = sp.sparse.hstack([
    vect1.transform(X2_test["description"]),
    vect2.transform(X2_test["designation"]),
    vect3.transform(X2_test["title"]),  
])

print("Score on test", LR.score(BoW_test,y_test))

Score on test 0.7194413333623249


We observe that the Bag of Word method on the text features performs much better than the Baseline model. We achieve 72% accuracy on both train and test.

## **Question 1.2.2**
Try using n-grams, characters, tf-idf rescaling and possibly other ways to tune the BoW model. Be aware that you might need to adjust the (regularization of the) linear model for different feature sets.


### **TF-IDF**
First let's have a look at the Tf-idf process

In [0]:
cat_BoW = ["description", "designation", "title"]
X3 = X_train[cat_BoW]
X3 = X3.fillna('missing')

X3_test = X_test[cat_BoW]
X3_test = X3_test.fillna('missing')

In [0]:
tfidf1 = make_pipeline(CountVectorizer(min_df=2, max_features=2000), TfidfTransformer())
tfidf2 = make_pipeline(CountVectorizer(min_df=2, max_features=1000, max_df=0.95), TfidfTransformer())
tfidf3 = make_pipeline(CountVectorizer(min_df=2, max_features=1000, max_df=0.95), TfidfTransformer())

BoW_train = sp.sparse.hstack([
    tfidf1.fit_transform(X3["description"]),
    tfidf2.fit_transform(X3["designation"]),
    tfidf3.fit_transform(X3["title"]),  
])

BoW_train

<97478x4000 sparse matrix of type '<class 'numpy.float64'>'
	with 3755908 stored elements in COOrdinate format>

In [0]:
BoW_sub, _, y_sub, _ = train_test_split(BoW_train, y_train, train_size=0.25)

param_grid_LR = {'alpha': np.logspace(-2,1, num=20)}  

grid_LR = GridSearchCV(Ridge(), param_grid_LR, cv=5, return_train_score=True)
grid_LR.fit(BoW_sub, y_sub)
print("Best mean cross-validation score for LR: {:.3f}".format(grid_LR.best_score_))
print("Best parameters: {}".format(grid_LR.best_params_))

Best mean cross-validation score for LR: 0.682
Best parameters: {'alpha': 1.1288378916846884}


In [0]:
LR = Ridge(alpha=1.129)
scores = cross_val_score(LR, BoW_train, y_train)
LR.fit(BoW_train, y_train)
print("Score Linear Regression", np.mean(scores))

Score Linear Regression 0.7245366236556758


The Tfidf process with a Ridge Regression gives a slightly better result than the Bag of Word baseline we previously did. It is therefore an improvement we keep for the other methods.

### **N-grams**
We will now use Bag of words on N-grams.

In [0]:
vect1 = make_pipeline(CountVectorizer(min_df=2, max_features=2000, ngram_range=(1,4)), TfidfTransformer())
vect2 = make_pipeline(CountVectorizer(min_df=2, max_features=1000, max_df=0.95, ngram_range=(1,4)), TfidfTransformer())
vect3 = make_pipeline(CountVectorizer(min_df=2, max_features=1000, max_df=0.95, ngram_range=(1,4)), TfidfTransformer())

BoW_train = sp.sparse.hstack([
    vect1.fit_transform(X3["description"]),
    vect2.fit_transform(X3["designation"]),
    vect3.fit_transform(X3["title"]),  
])

BoW_train

<97478x4000 sparse matrix of type '<class 'numpy.float64'>'
	with 5012906 stored elements in COOrdinate format>

In [0]:
BoW_sub, _, y_sub, _ = train_test_split(BoW_train, y_train, train_size=0.25)

param_grid_LR = {'alpha': np.logspace(-2,1, num=20)}  

grid_LR = GridSearchCV(Ridge(), param_grid_LR, cv=5, return_train_score=True)
grid_LR.fit(BoW_sub, y_sub)
print("Best mean cross-validation score for LR: {:.3f}".format(grid_LR.best_score_))
print("Best parameters: {}".format(grid_LR.best_params_))

Best mean cross-validation score for LR: 0.665
Best parameters: {'alpha': 2.3357214690901213}


In [0]:
LR = Ridge(alpha=2.336)
scores = cross_val_score(LR, BoW_train, y_train)
LR.fit(BoW_train, y_train)
print("Score Linear Regression", np.mean(scores))

Score Linear Regression 0.6983278147144512


Using N-grams on words performs worse than classical Tfidf, we will therefore not keep this method for further analysis.

### **Characters**
Let's have a look at N-grams using characters.

In [0]:
vect1 = make_pipeline(CountVectorizer(min_df=3, max_features=2000, max_df=0.95, ngram_range=(3,7), analyzer='char_wb'), TfidfTransformer())
vect2 = make_pipeline(CountVectorizer(min_df=3, max_features=1000, max_df=0.95, ngram_range=(3,7), analyzer='char_wb'), TfidfTransformer())
vect3 = make_pipeline(CountVectorizer(min_df=3, max_features=1000, max_df=0.95, ngram_range=(3,7), analyzer='char_wb'), TfidfTransformer())

BoW_train = sp.sparse.hstack([
    vect1.fit_transform(X3["description"]),
    vect2.fit_transform(X3["designation"]),
    vect3.fit_transform(X3["title"]),  
])

BoW_train

<97478x4000 sparse matrix of type '<class 'numpy.float64'>'
	with 36321105 stored elements in COOrdinate format>

In [0]:
BoW_sub, _, y_sub, _ = train_test_split(BoW_train, y_train, train_size=0.25)

param_grid_LR = {'alpha': np.logspace(-2,1, num=20)}  

grid_LR = GridSearchCV(Ridge(), param_grid_LR, cv=5, return_train_score=True)
grid_LR.fit(BoW_sub, y_sub)
print("Best mean cross-validation score for LR: {:.3f}".format(grid_LR.best_score_))
print("Best parameters: {}".format(grid_LR.best_params_))

Best mean cross-validation score for LR: 0.654
Best parameters: {'alpha': 0.7847599703514611}


In [0]:
LR = Ridge(alpha=0.785)
scores = cross_val_score(LR, BoW_train, y_train)
LR.fit(BoW_train, y_train)
print("Score Linear Regression", np.mean(scores))

Score Linear Regression 0.6777995896672605


Using N-grams on characters performs similarly to N-grams on words. However, the characters perform worse than classical Tfidf. We will not use this method for further analysis.

In conclusion, Using Tf-idf yields the best score out of the Bag of words embedding methods.


## **Question 1.3**
Combine the non-text features and the text features. How does adding those features improve upon just using bag-of-words?


In [0]:
cat_BoW = ["description", "designation", "title"]
cont = ["price"]
cat_TE = ["province", "region_1", "variety", "winery"]
cat_OHE = ["country", "region_2", "taster_name", "taster_twitter_handle"]

X4 = X_train[cat_BoW + cont + cat_OHE + cat_TE]
X4[cat_BoW + cat_OHE + cat_TE] = X4[cat_BoW + cat_OHE + cat_TE].fillna('missing')
Cont_Imputer = SimpleImputer(strategy='median')
X4[cont] = Cont_Imputer.fit_transform(X4[cont])

X4_test = X_test[cat_BoW + cont + cat_OHE + cat_TE]
X4_test[cat_BoW + cat_OHE + cat_TE] = X4_test[cat_BoW + cat_OHE + cat_TE].fillna('missing')
X4_test[cont] = Cont_Imputer.transform(X4_test[cont])

In [0]:
tfidf1 = make_pipeline(CountVectorizer(min_df=2, max_features=2000), TfidfTransformer())
tfidf2 = make_pipeline(CountVectorizer(min_df=2, max_features=1000, max_df=0.95), TfidfTransformer())
tfidf3 = make_pipeline(CountVectorizer(min_df=2, max_features=1000, max_df=0.95), TfidfTransformer())
preprocess = make_column_transformer(
    (TargetEncoder(), cat_TE),
    (OneHotEncoder(handle_unknown='ignore'), cat_OHE),
    (StandardScaler(), cont)
)

new_train = sp.sparse.hstack([
    tfidf1.fit_transform(X4["description"]),
    tfidf2.fit_transform(X4["designation"]),
    tfidf3.fit_transform(X4["title"]),
    preprocess.fit_transform(X4[cont + cat_OHE + cat_TE], y_train),
])

new_test = sp.sparse.hstack([
    tfidf1.transform(X4_test["description"]),
    tfidf2.transform(X4_test["designation"]),
    tfidf3.transform(X4_test["title"]),
    preprocess.transform(X4_test[cont + cat_OHE + cat_TE]),
])

new_train

<97478x4102 sparse matrix of type '<class 'numpy.float64'>'
	with 4633210 stored elements in COOrdinate format>

First we try a Ridge Regression on the combination looking for alpha by doing a grid search.

In [0]:
new_sub, _, y_sub, _ = train_test_split(new_train, y_train, train_size=0.35)

param_grid_LR = {'alpha': np.logspace(-2,1, num=30)}  

grid_LR = GridSearchCV(Ridge(), param_grid_LR, cv=5, return_train_score=True)
grid_LR.fit(new_sub, y_sub)
print("Best mean cross-validation score for LR: {:.3f}".format(grid_LR.best_score_))
print("Best parameters: {}".format(grid_LR.best_params_))

Best mean cross-validation score for LR: 0.760
Best parameters: {'alpha': 1.4873521072935119}


In [0]:
LR = Ridge(alpha=1.4873521072935119)

scores = cross_val_score(LR, new_train, y_train)
LR.fit(new_train, y_train)
print("Score Ridge Regression", np.mean(scores))

Score Ridge Regression 0.7785811478130501


Combining text and non-text features with a Ridge model yields our best score yet. The score is better than the one we obtained using only BoW and Tfidf. Here we have almost 78% on the train dataset.
<br>
Nonetheless will compare this model to one using Linear Regression.

In [0]:
model_LR = LinearRegression()

scores = cross_val_score(model_LR, new_train, y_train)
model_LR.fit(new_train, y_train)
print("Score Linear Regression", np.mean(scores))

Score Linear Regression 0.7795396893657197


In [0]:
model_LR.score(new_test,y_test)

0.7585818844504174

The classical Linear Regression performs slightly better than the Ridge. We will keep it as our final model. In conclusion we achieved almost 78% on the train dataset and 76% on the test dataset by using Tf-idf Bag of words embedding on the text features and taking into account the non-text features.